We'll start our PowerCLI demos with a comparison of the .NET object returned from the PowerCLI cmdlets such as `Get-VM`, and the vSphere object returned from `Get-View`.

This entire vSphere complex object is also contained within the "ExtensionData" property of the .NET object returned from `Get-VM`.

This is unknown to many PowerCLI users, but this is a significant part of why getting the .NET object takes so long to get.

Let's start with connecting to our environment.

In [2]:
$vCenterServer = 'flashstack-vcenter.puretec.purestorage.com'
$vCenterUser= 'administrator@vsphere.local'
$vCenterCred = Get-Credential -UserName $vCenterUser
Connect-VIServer -Server $vCenterServer -Credential $vCenterCred -Protocol https -ErrorAction Stop

Now that we're connected, see how long it takes for us to use `Get-VM` to return a .NET object of a virtual machine.

We will follow this by piping our virtual machine object to `Get-Snapshot` to get the snapshots for all of our virtual machines.

In [3]:
$Stopwatch = [System.Diagnostics.Stopwatch]::new()
$Stopwatch.Start()

$OverheadVMs = Get-VM
$OverheadVMs | Get-Snapshot | Select-Object -Property VM, Name

$Stopwatch.Stop()
$Stopwatch.Elapsed.TotalSeconds


VM                    Name
--                    ----
orc-node2             os-install
orc-node2             oracleasmsupport
orc-node2             before useradd
orc-node2             before nvme
orc-node2             after NVMe setup
orc-node2             P1
orc-node2             p2
orc-node1             os-install
orc-node1             oracleasmsupport
orc-node1             before useradd
orc-node1             before nvme
orc-node1             after NVMe setup
orc-node1             before grid install
orc-node1             P1
orc-node1             p2
postgresdb-test       OS install
postgresdb-test       after ansible
postgresdb-test       does successful
fs-victoriametricslab before vmagent
fs-prometheuslab      before
fs-prometheuslab      before fdisk
fs-prometheuslab      after prometheus
fs-prometheuslab      after grafana
fs-prometheuslab      after exporter
fs-prometheuslab      before nodeexporter
fs-prometheuslab      before pgexporter
FlashStack-VC         Recovered - 4%2

Now that we have our .NET object, we will use `Get-View` to return a .NET object of a virtual machine.

Since this vSphere object contains the properties for our snapshot information of the virtual machine, we can get our snapshot details without the costly pipeline operation and querying with a second cmdlet.

In [4]:
$Stopwatch = [System.Diagnostics.Stopwatch]::new()
$Stopwatch.Start()

$FastVMs = Get-View -ViewType VirtualMachine
$FastVMs | Select-Object -Property Name, `
@{n = 'Snapshot'; e = { $PSItem.Snapshot.RootSnapshotList[0].Name } } | `
Where-Object Snapshot

$Stopwatch.Stop()
$Stopwatch.Elapsed.TotalSeconds


Name                  Snapshot
----                  --------
intersight-app        pre-install
fslab-ad              4april
FS-WinJmp1            4April
postgresdb-test       OS install
fs-vidm               4April
fs-vra                4april
fs-vrlscm             4April
rheltemplate-fslab    rheltemplate-fslab_vm-1876_1
FlashStack-VC         Recovered - 4%2f12%2f2023, 1:57:42 PM
fs-linjmp01           ansible before
fs-prometheuslab      before
fs-victoriametricslab before vmagent
orc-node1             os-install
orc-node2             os-install
0.712396

